---
title: Módulo 7
subtitle: Deep Learning
author:
  - name: Eloy Alvarado Narváez
    orcid: 0000-0001-7522-2327
    email: eloy.alvarado@usm.cl
    affiliations: Universidad Técnica Federico Santa María
  - name: Esteban Salgado Valenzuela
    orcid: 0000-0002-7799-0044
    affiliations: Universidad Técnica Federico Santa María
date: 12/13/2024
---

Utilizamos el paquete `torch` de **Python**, junto con el paquete `pytorch_lightning`, que proporciona utilidades para simplificar el ajuste y la evaluación de modelos. El paquete está bien estructurado, es flexible y resultará cómodo para los usuarios familiarizados con **Python**.

Un buen recurso complementario es el sitio [pytorch.org/tutorials](https://pytorch.org/tutorials/beginner/basics/intro.html). 

Comenzamos cargando varias librerias como en los laboratorios pasados:

In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
     (LinearRegression,
      LogisticRegression,
      Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import \
     (train_test_split,
      GridSearchCV)

### Librerías específicas de Torch

Hay varias importaciones necesarias para `torch`. Primero, importamos la biblioteca principal y las herramientas esenciales utilizadas para especificar redes estructuradas secuencialmente.


In [2]:
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset


Hay varios otros paquetes auxiliares para `torch`. Por ejemplo: 
 
- El paquete **`torchmetrics`** tiene utilidades para calcular diversas métricas y evaluar el rendimiento al ajustar un modelo.  
- El paquete **`torchinfo`** proporciona un resumen útil de las capas de un modelo.  

Usamos la función `read_image()` al cargar imágenes de prueba.  

In [3]:
from torchmetrics import (MeanAbsoluteError,
                          R2Score)
from torchinfo import summary


El paquete **`pytorch_lightning`** es una interfaz de nivel algo más alto para **`torch`** que simplifica la especificación y el ajuste de modelos, al reducir la cantidad de **código repetitivo** necesario (en comparación con usar `torch` por sí solo).


In [4]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger


Para reproducir resultados, usamos `seed_everything()`. También indicaremos a **`torch`** que utilice **algoritmos deterministas** siempre que sea posible.

In [5]:
from pytorch_lightning import seed_everything
seed_everything(0, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)


Utilizaremos varios conjuntos de datos incluidos con **`torchvision`** para el laboratorio: una red preentrenada para clasificación de imágenes, así como algunas **transformaciones** utilizadas para el preprocesamiento.

In [6]:
from torchvision.io import read_image
from torchvision.datasets import MNIST, CIFAR100
from torchvision.models import (resnet50,
                                ResNet50_Weights)
from torchvision.transforms import (Resize,
                                    Normalize,
                                    CenterCrop,
                                    ToTensor)

- **`SimpleDataModule`** y **`SimpleModule`** son versiones simplificadas de objetos utilizados en **`pytorch_lightning`**, el módulo de alto nivel para ajustar modelos de **`torch`**.  

Aunque es posible realizar usos más avanzados, como computación en **unidades de procesamiento gráfico (GPUs)** y procesamiento paralelo de datos en este módulo, no nos enfocaremos mucho en estos temas en este laboratorio.  

- **`ErrorTracker`** maneja colecciones de objetivos (*targets*) y predicciones en cada **mini-batch** durante las etapas de validación o prueba, permitiendo calcular la métrica en todo el conjunto de datos de validación o prueba.


In [7]:
from ISLP.torch import (SimpleDataModule,
                        SimpleModule,
                        ErrorTracker,
                        rec_num_workers)


Los datos preprocesados de **`IMDb`** son provenientes de **`keras`**, un paquete independiente para ajustar modelos de aprendizaje profundo.  

Esto nos ahorra una cantidad significativa de **preprocesamiento** y nos permite enfocarnos en la especificación y ajuste de los propios modelos.


In [8]:
from ISLP.torch.imdb import (load_lookup,
                             load_tensor,
                             load_sparse,
                             load_sequential)


Finalmente, introducimos algunas **importaciones auxiliares** que no están directamente relacionadas con **`torch`**.  

- La función **`glob()`** del módulo **`glob`** se utiliza para encontrar todos los archivos que coinciden con caracteres **comodín** (*wildcards*). La usaremos en nuestro ejemplo aplicando el modelo **`ResNet50`** a algunas de nuestras propias imágenes.  
- El módulo **`json`** se utilizará para cargar un archivo **JSON** que nos permitirá buscar las clases y **identificar las etiquetas** de las imágenes en el ejemplo de **`ResNet50`**.  


In [9]:
from glob import glob
import json


## Red de Capa Única en los Datos de Hitters  

Comenzamos ajustando modelos utilizando los datos de **`Hitters`**.


In [10]:
Hitters = load_data('Hitters').dropna()
n = Hitters.shape[0]
Hitters.head()


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A


Ajustaremos dos modelos lineales (**mínimos cuadrados** y **lasso**) y compararemos su rendimiento con el de una **red neuronal**. Para esta comparación utilizaremos el **error absoluto medio (MAE)** en un conjunto de datos de validación:

$$
\text{MAE}(y, \hat{y}) = \frac{1}{n} \sum_{i=1}^n |y_i - \hat{y}_i|.
$$

Configuramos la matriz de características y la variable de respuesta.


In [11]:
model = MS(Hitters.columns.drop('Salary'), intercept=False)
X = model.fit_transform(Hitters).to_numpy()
Y = Hitters['Salary'].to_numpy()


El método **`to_numpy()`** convierte **data frames** o **series** de `pandas` en arreglos de **`numpy`**.  
Hacemos esto porque necesitaremos usar **`sklearn`** para ajustar el modelo **lasso**, y esta librería requiere dicha conversión.

También utilizamos un método de **regresión lineal** de `sklearn`, en lugar del método de **`statsmodels`** para **facilitar las comparaciones**.


Ahora dividimos los datos en **entrenamiento** y **prueba**, fijando el **estado aleatorio** utilizado por `sklearn` para realizar la división.


In [12]:
(X_train, 
 X_test,
 Y_train,
 Y_test) = train_test_split(X,
                            Y,
                            test_size=1/3,
                            random_state=1)

### Modelos Lineales  
Ajustamos el modelo lineal y evaluamos directamente el error en el conjunto de prueba.


In [13]:
hit_lm = LinearRegression().fit(X_train, Y_train)
Yhat_test = hit_lm.predict(X_test)
np.abs(Yhat_test - Y_test).mean()

259.7152883314631

A continuación ajustamos el modelo **lasso** utilizando **`sklearn`**. Usamos el **error absoluto medio (MAE)** para seleccionar y evaluar el modelo, en lugar del **error cuadrático medio (MSE)**.  

Aquí creamos una **malla de validación cruzada** y la ejecutamos directamente.

Codificamos un **pipeline** con dos pasos:  
1. Primero **normalizamos** las características utilizando un transformador **`StandardScaler()`**.  
2. Luego ajustamos el modelo **lasso** sin realizar una normalización adicional.


In [14]:
scaler = StandardScaler(with_mean=True, with_std=True)
lasso = Lasso(warm_start=True, max_iter=30000)
standard_lasso = Pipeline(steps=[('scaler', scaler),
                                 ('lasso', lasso)])

Necesitamos crear una **malla de valores** para $\lambda$. Como es común, elegimos una malla de **100 valores** de $\lambda$, distribuidos de manera uniforme en la **escala logarítmica** desde `lam_max` hasta `0.01 \cdot \text{lam_max}`.  

Aquí, `lam_max` es el valor más pequeño de $\lambda$ que produce una **solución completamente cero**. Este valor es igual al **mayor producto interno absoluto** entre cualquier predictor y la respuesta (centrada).  
{La derivación de este resultado está fuera del alcance de este libro.}


In [15]:
X_s = scaler.fit_transform(X_train)
n = X_s.shape[0]
lam_max = np.fabs(X_s.T.dot(Y_train - Y_train.mean())).max() / n
param_grid = {'lasso__alpha': np.exp(np.linspace(0, np.log(0.01), 100))
             * lam_max}

Es importante notar que tuvimos que **transformar los datos** primero, ya que la escala de las variables impacta la elección de $\lambda$.  

Ahora realizamos **validación cruzada** utilizando esta secuencia de valores de $\lambda$.


In [16]:
cv = KFold(10,
           shuffle=True,
           random_state=1)
grid = GridSearchCV(standard_lasso,
                    param_grid,
                    cv=cv,
                    scoring='neg_mean_absolute_error')
grid.fit(X_train, Y_train);

Extraemos el modelo **lasso** con el **menor error absoluto medio** validado cruzadamente y evaluamos su rendimiento en `X_test` y `Y_test`, que no fueron utilizados en la validación cruzada.


In [17]:
trained_lasso = grid.best_estimator_
Yhat_test = trained_lasso.predict(X_test)
np.fabs(Yhat_test - Y_test).mean()

235.6754837478029

Esto es similar a los resultados obtenidos para el modelo lineal ajustado por **mínimos cuadrados**. Sin embargo, estos resultados pueden variar considerablemente con diferentes divisiones de entrenamiento/prueba.

### Especificación de una Red: Clases y Herencia

Para ajustar la **red neuronal**, primero configuramos una **estructura de modelo** que describe la red.  
Esto requiere que definamos **nuevas clases** específicas para el modelo que deseamos ajustar.  

Típicamente, esto se hace en **`pytorch`** sub-clasificando una representación genérica de una red, que es el enfoque que tomamos aquí.  

Aunque este ejemplo es **simple**, repasaremos los pasos con cierto detalle, ya que esto nos será útil para el resto de ejemplos.


In [18]:
class HittersModel(nn.Module):

    def __init__(self, input_size):
        super(HittersModel, self).__init__()
        self.flatten = nn.Flatten()
        self.sequential = nn.Sequential(
            nn.Linear(input_size, 50),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(50, 1))

    def forward(self, x):
        x = self.flatten(x)
        return torch.flatten(self.sequential(x))


La declaración **`class`** identifica el bloque de código como una **declaración de clase** llamada `HittersModel`, que hereda de la clase base **`nn.Module`**. Esta clase base es **ampliamente utilizada** en **`torch`** y representa las **mapeos** en las redes neuronales.

Indentados bajo la declaración de la clase están los **métodos** de esta clase:  
en este caso, **`__init__`** y **`forward`**.  

- El método **`__init__`** es llamado cuando se crea una instancia de la clase, como se muestra en la celda a continuación.  
- En los métodos, **`self`** siempre hace referencia a una instancia de la clase.  

En el método **`__init__`**, hemos adjuntado dos objetos a `self` como atributos:  
- **`flatten`**  
- **`sequential`**  

Estos atributos se utilizan en el método **`forward`** para describir el **mapeo** que implementa este módulo.

### Uso de `super()`

Hay una línea adicional en el método **`__init__`**, que es una llamada a **`super()`**.  

- Esta función permite a las **subclases** (es decir, `HittersModel`) acceder a los métodos de la clase que heredan.  
- Por ejemplo, la clase **`nn.Module`** tiene su propio método `__init__`, que es diferente del método **`HittersModel.__init__()`** que hemos escrito.  

Usar **`super()`** nos permite llamar al método de la clase base.  
Para los modelos de **`torch`**, siempre haremos esta llamada a **`super()`** ya que es **necesaria** para que el modelo sea correctamente interpretado por **`torch`**.

### Métodos adicionales de `nn.Module`

El objeto **`nn.Module`** tiene más métodos además de **`__init__`** y **`forward`**. Estos métodos son **directamente accesibles** a instancias de `HittersModel` debido a la **herencia**.

Uno de estos métodos, es el método **`eval()`**, que se utiliza para **desactivar dropout** cuando queremos evaluar el modelo en **datos de prueba**.


In [19]:
hit_model = HittersModel(X.shape[1])


El objeto **`self.sequential`** es una composición de **cuatro mapeos**.  

1. El primer mapeo lleva las **19 características** de **`Hitters`** a **50 dimensiones**, introduciendo $50 \times 19 + 50$ parámetros para los pesos y el **intercepto** del mapeo (a menudo llamado *bias*).  
2. Esta capa se mapea a una capa **ReLU**.  
3. Luego es seguida por una capa de **dropout del 40%**.  
4. Finalmente, se aplica un mapeo lineal a **1 dimensión**, nuevamente con un *bias*.  

El número total de parámetros entrenables es:  

$$
50 \times 19 + 50 + 50 + 1 = 1051.
$$


El paquete **`torchinfo`** proporciona la función **`summary()`** que resume esta información de manera ordenada.  

Podemos especificar el **tamaño de la entrada** y ver el tamaño de cada **tensor** a medida que pasa por las capas de la red.

In [20]:
summary(hit_model, 
        input_size=X_train.shape,
        col_names=['input_size',
                   'output_size',
                   'num_params'])


/home/ealvnrz/.pyenv/versions/3.10.5/lib/python3.10/site-packages/torch/nn/modules/linear.py:125: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:208.)
  return F.linear(input, self.weight, self.bias)


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
HittersModel                             [175, 19]                 [175]                     --
├─Flatten: 1-1                           [175, 19]                 [175, 19]                 --
├─Sequential: 1-2                        [175, 19]                 [175, 1]                  --
│    └─Linear: 2-1                       [175, 19]                 [175, 50]                 1,000
│    └─ReLU: 2-2                         [175, 50]                 [175, 50]                 --
│    └─Dropout: 2-3                      [175, 50]                 [175, 50]                 --
│    └─Linear: 2-4                       [175, 50]                 [175, 1]                  51
Total params: 1,051
Trainable params: 1,051
Non-trainable params: 0
Total mult-adds (M): 0.18
Input size (MB): 0.01
Forward/backward pass size (MB): 0.07
Params size (MB): 0.00
Estimated Total Size (MB): 0.09

Ahora necesitamos transformar nuestros datos de entrenamiento en una forma accesible para **`torch`**.  
El tipo de dato básico en **`torch`** es un **`tensor`**.

También notamos que **`torch`** típicamente trabaja con números de punto flotante de **32 bits** (*precisión simple*) en lugar de **64 bits** (*precisión doble*).  
Por lo tanto, convertimos nuestros datos a **`np.float32`** antes de formar el tensor.

Los tensores $X$ y $Y$ se organizan luego en un **`Dataset`** reconocido por **`torch`** utilizando **`TensorDataset()`**.

In [21]:
X_train_t = torch.tensor(X_train.astype(np.float32))
Y_train_t = torch.tensor(Y_train.astype(np.float32))
hit_train = TensorDataset(X_train_t, Y_train_t)

Hacemos lo mismo para el conjunto de prueba

In [22]:
X_test_t = torch.tensor(X_test.astype(np.float32))
Y_test_t = torch.tensor(Y_test.astype(np.float32))
hit_test = TensorDataset(X_test_t, Y_test_t)


Finalmente, este **dataset** se pasa a un **`DataLoader()`**, que en última instancia entrega los datos a nuestra red. Aunque esto puede parecer **excesivamente complejo**, esta estructura es útil para tareas más **avanzadas**, donde los datos pueden residir en diferentes **máquinas** o deben pasarse a una **GPU**.

- Uno de sus argumentos es **`num_workers`**, que indica cuántos procesos utilizaremos para cargar los datos.  
- Para datos pequeños como **`Hitters`**, esto tendrá poco efecto.  
- Sin embargo, ofrece una **ventaja** para los ejemplos de **`MNIST`** y **`CIFAR100`** presentados más adelante.

El paquete **`torch`** inspeccionará el proceso en ejecución y determinará el **número máximo de trabajadores**.  
(Esto depende del **hardware de cómputo** y del número de **núcleos disponibles**.)

Hemos incluido una función **`rec_num_workers()`** para calcular esto y saber cuántos trabajadores podrían ser razonables (en este caso, el máximo fue **16**).


In [23]:
max_num_workers = rec_num_workers()

La configuración general de **entrenamiento** en **`pytorch_lightning`** involucra datos de **entrenamiento**, **validación** y **prueba**.  
Estos se representan mediante diferentes **data loaders**.  

- Durante cada **epoch**, ejecutamos un **paso de entrenamiento** para aprender el modelo y un **paso de validación** para rastrear el error.  
- Los datos de prueba se utilizan típicamente al final del entrenamiento para **evaluar el modelo**.

En este caso, como sólo dividimos los datos en **entrenamiento** y **prueba**, utilizaremos los datos de **prueba** como datos de **validación** utilizando el argumento `validation=hit_test`.

- El argumento **`validation`** puede ser:  
   - Un **float** entre **0** y **1**: interpretado como un **porcentaje** de las observaciones de **entrenamiento** a usar para validación.  
   - Un **entero**: interpretado como el **número** de observaciones de **entrenamiento** a usar para validación.  
   - Un **`Dataset`**: se pasa directamente a un **data loader**.  


In [24]:
hit_dm = SimpleDataModule(hit_train,
                          hit_test,
                          batch_size=32,
                          num_workers=min(4, max_num_workers),
                          validation=hit_test)


A continuación, debemos proporcionar un módulo de **`pytorch_lightning`** que controle los pasos realizados durante el proceso de entrenamiento.  

Proporcionamos métodos para nuestro **`SimpleModule()`** que simplemente registran el valor de la **función de pérdida** y cualquier **métrica adicional** al final de cada época.  

Estas operaciones son controladas por los métodos:  
- `SimpleModule.training_step()`  
- `SimpleModule.test_step()`  
- `SimpleModule.validation_step()`  

Sin embargo, **no modificaremos** estos métodos en nuestros ejemplos.


In [25]:
hit_module = SimpleModule.regression(hit_model,
                           metrics={'mae':MeanAbsoluteError()})


Al usar el método **`SimpleModule.regression()`**, indicamos que utilizaremos la **pérdida de error cuadrático**. También solicitamos que se registre el **error absoluto medio (MAE)** dentro de las métricas que se **loggean**.

Registramos nuestros resultados mediante **`CSVLogger()`**, que en este caso almacena los resultados en un archivo **CSV** dentro del directorio **`logs/hitters`**.  

Una vez que el ajuste está completo, esto nos permite cargar los resultados como un **`pd.DataFrame()`** y visualizarlos a continuación.  

Existen varias maneras de registrar los resultados en **`pytorch_lightning`**, aunque no cubriremos esos detalles aquí.

In [26]:
hit_logger = CSVLogger('logs', name='hitters')

Finalmente, estamos listos para **entrenar nuestro modelo** y registrar los resultados. Utilizamos el objeto **`Trainer()`** de **`pytorch_lightning`** para realizar este trabajo.  

- El argumento **`datamodule=hit_dm`** le dice al **trainer** cómo producir los **logs** de entrenamiento/validación/prueba.  
- El primer argumento, **`hit_module`**, especifica la **arquitectura de la red** así como los pasos de entrenamiento, validación y prueba.  
- El argumento **`callbacks`** permite llevar a cabo varias tareas en diferentes puntos durante el **entrenamiento** del modelo.  

Aquí, nuestro **callback `ErrorTracker()`** nos permitirá calcular el **error de validación** durante el entrenamiento y, finalmente, el **error de prueba**.  

Ahora ajustamos el modelo durante **50 épocas**.


In [27]:
hit_trainer = Trainer(deterministic=True,
                      max_epochs=50,
                      log_every_n_steps=5,
                      logger=hit_logger,
                      callbacks=[ErrorTracker()])
hit_trainer.fit(hit_module, datamodule=hit_dm)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility

En cada paso del **SGD**, el algoritmo selecciona aleatoriamente **32 observaciones** de entrenamiento para calcular el gradiente.  

Recordemos, que una **época** equivale al número de pasos de **SGD** necesarios para procesar $n$ observaciones.  

Dado que el conjunto de entrenamiento tiene $n=175$, y especificamos un **`batch_size`** de 32 en la construcción de **`hit_dm`**, una **época** corresponde a:  

$$
175 / 32 = 5.5 \ \text{pasos de SGD}.
$$

Después de ajustar el modelo, podemos evaluar el rendimiento en nuestros datos de prueba utilizando el método **`test()`** de nuestro **trainer**.


In [28]:
hit_trainer.test(hit_module, datamodule=hit_dm)


Testing: |          | 0/? [00:00<?, ?it/s]

RuntimeError: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility

Los resultados del ajuste se han guardado en un **archivo CSV**. Podemos encontrar los resultados específicos de esta ejecución en el atributo **`experiment.metrics_file_path`** de nuestro **logger**.  

Es importante notar que cada vez que el modelo se ajusta, el **logger** guarda los resultados en un **nuevo subdirectorio** dentro del directorio **`logs/hitters`**.

Ahora creamos una **gráfica del MAE** (**error absoluto medio**) como función del **número de épocas**.  
Primero, recuperamos los **resúmenes registrados**.


In [29]:
hit_results = pd.read_csv(hit_logger.experiment.metrics_file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'logs/hitters/version_1/metrics.csv'

Dado que produciremos gráficos similares en ejemplos posteriores, escribimos una **función genérica simple** para generar este gráfico.


In [30]:
def summary_plot(results,
                 ax,
                 col='loss',
                 valid_legend='Validation',
                 training_legend='Training',
                 ylabel='Loss',
                 fontsize=20):
    # Extraer datos manualmente para evitar errores con pandas.plot
    epochs = results['epoch']
    train_values = results[f'train_{col}_epoch']
    valid_values = results[f'valid_{col}']
    
    # Graficar las curvas manualmente
    ax.plot(epochs, train_values, marker='o', color='black', label=training_legend)
    ax.plot(epochs, valid_values, marker='o', color='red', label=valid_legend)

    # Etiquetas y leyenda
    ax.set_xlabel('Epoch', fontsize=fontsize)
    ax.set_ylabel(ylabel, fontsize=fontsize)
    ax.legend(loc='best', fontsize=fontsize * 0.8)
    
    return ax


Ahora configuramos nuestros **ejes** y utilizamos nuestra función para generar el gráfico del **MAE**.

In [31]:
fig, ax = subplots(1, 1, figsize=(6, 6))
ax = summary_plot(hit_results,
                  ax,
                  col='mae',
                  ylabel='MAE',
                  valid_legend='Validation (=Test)')

ax.set_ylim([0, 400])
ax.set_xticks(np.linspace(0, 50, 11).astype(int))
plt.show()


NameError: name 'hit_results' is not defined

Podemos realizar **predicciones** directamente a partir del modelo final y evaluar su rendimiento en los datos de prueba.  

Antes de predecir, llamamos al método **`eval()`** de **`hit_model`**.  
Esto le indica a **`torch`** que considere efectivamente este modelo como **ajustado**, de modo que podamos utilizarlo para predecir en nuevos datos.  

Para nuestro modelo aquí, el cambio más significativo es que las **capas de dropout** se **desactivarán**, es decir, no se eliminarán pesos de manera aleatoria al predecir en nuevos datos.


In [ ]:
hit_model.eval() 
preds = hit_module(X_test_t)
torch.abs(Y_test_t - preds).mean()

### Limpieza  

Al configurar nuestro **módulo de datos**, habíamos iniciado varios **procesos de trabajo** (*worker processes*) que permanecerán en ejecución.  

Eliminamos todas las referencias a los objetos de **`torch`** para asegurarnos de que estos procesos sean terminados.


In [32]:
del(Hitters,
    hit_model, hit_dm,
    hit_logger,
    hit_test, hit_train,
    X, Y,
    X_test, X_train,
    Y_test, Y_train,
    X_test_t, Y_test_t,
    hit_trainer, hit_module)


## Red Multicapa en los Datos de Dígitos MNIST  

El paquete **`torchvision`** incluye una serie de conjuntos de datos de ejemplo, entre ellos los datos de dígitos **`MNIST`**.  

Nuestro primer paso es recuperar los conjuntos de datos de **entrenamiento** y **prueba**. La función **`MNIST()`** dentro de **`torchvision.datasets`** se proporciona para este propósito.  

Los datos se descargarán la **primera vez** que esta función sea ejecutada, y se almacenarán en el directorio **`data/MNIST`**.


In [33]:
(mnist_train, 
 mnist_test) = [MNIST(root='data',
                      train=train,
                      download=True,
                      transform=ToTensor())
                for train in [True, False]]
mnist_train


Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

Hay **60,000 imágenes** en los datos de **entrenamiento** y **10,000 imágenes** en los datos de **prueba**. Las imágenes tienen un tamaño de **$28 \times 28$** y se almacenan como una **matriz de píxeles**. Necesitamos transformar cada imagen en un **vector**.

Las redes neuronales son **sensibles a la escala** de las entradas. Aquí, las entradas son valores **grayscale de 8 bits**, que van de **0 a 255**, por lo que las reescalamos al **intervalo unitario**.  

Esta transformación, junto con el **reordenamiento de los ejes**, se realiza mediante la función **`ToTensor()`** del paquete **`torchvision.transforms`**.

Como en nuestro ejemplo con el conjunto de datos **`Hitters`**, formamos un **módulo de datos** a partir de los conjuntos de datos de **entrenamiento** y **prueba**, reservando el **20%** de las imágenes de entrenamiento para **validación**.


In [34]:
mnist_dm = SimpleDataModule(mnist_train,
                            mnist_test,
                            validation=0.2,
                            num_workers=max_num_workers,
                            batch_size=256)


Revisando los datos que se alimentarán a nuestra red. Recorremos los primeros **lotes** del conjunto de datos de prueba, deteniéndonos después de **2 lotes**:


In [35]:
for idx, (X_ ,Y_) in enumerate(mnist_dm.train_dataloader()):
    print('X: ', X_.shape)
    print('Y: ', Y_.shape)
    if idx >= 1:
        break


X:  torch.Size([256, 1, 28, 28])
Y:  torch.Size([256])
X:  torch.Size([256, 1, 28, 28])
Y:  torch.Size([256])


Observamos que $X$ para cada lote consiste en **256 imágenes** de tamaño **`1x28x28`**.  
Aquí, el **`1`** indica un **único canal** (escala de grises). Para imágenes **RGB**, como las de **`CIFAR100`** más adelante, el **`1`** en el tamaño será reemplazado por **`3`** para los tres canales RGB.

Ahora estamos listos para **especificar nuestra red neuronal**.


In [36]:
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Dropout(0.4))
        self.layer2 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3))
        self._forward = nn.Sequential(
            self.layer1,
            self.layer2,
            nn.Linear(128, 10))
    def forward(self, x):
        return self._forward(x)

Observamos que en la **primera capa**, cada imagen de tamaño **`1x28x28`** es **aplanada** (*flattened*) y luego mapeada a **256 dimensiones**, donde aplicamos una **activación ReLU** con **40% de dropout**.  

Una **segunda capa** reduce la salida de la primera capa a **128 dimensiones**, aplicando una **activación ReLU** con **30% de dropout**.  

Finalmente, las **128 dimensiones** se reducen a **10**, que es el número de **clases** en los datos de **`MNIST`**.


In [37]:
mnist_model = MNISTModel()


Podemos verificar que el modelo produce una salida del **tamaño esperado** utilizando nuestro lote existente **`X_`** mencionado anteriormente.

In [38]:
mnist_model(X_).size()

torch.Size([256, 10])

Revisemos **resumen del modelo**. En lugar de especificar un **`input_size`**, podemos pasar un **tensor** con la forma correcta.  

En este caso, utilizamos el lote final **`X_`** mencionado anteriormente.


In [39]:
summary(mnist_model,
        input_data=X_,
        col_names=['input_size',
                   'output_size',
                   'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
MNISTModel                               [256, 1, 28, 28]          [256, 10]                 --
├─Sequential: 1-1                        [256, 1, 28, 28]          [256, 10]                 --
│    └─Sequential: 2-1                   [256, 1, 28, 28]          [256, 256]                --
│    │    └─Flatten: 3-1                 [256, 1, 28, 28]          [256, 784]                --
│    │    └─Linear: 3-2                  [256, 784]                [256, 256]                200,960
│    │    └─ReLU: 3-3                    [256, 256]                [256, 256]                --
│    │    └─Dropout: 3-4                 [256, 256]                [256, 256]                --
│    └─Sequential: 2-2                   [256, 256]                [256, 128]                --
│    │    └─Linear: 3-5                  [256, 256]                [256, 128]                32,896
│    │    └─ReLU: 3-6     

Habiendo configurado tanto el **modelo** como el **módulo de datos**, ajustar este modelo es ahora casi **idéntico** al ejemplo de **`Hitters`**.  

A diferencia de nuestro modelo de regresión, aquí utilizaremos el método **`SimpleModule.classification()`**, el cual usa la función de pérdida de **entropía cruzada** en lugar del error cuadrático medio.  

Debe proporcionarse el **número de clases** en el problema.


In [40]:
mnist_module = SimpleModule.classification(mnist_model,
                                           num_classes=10)
mnist_logger = CSVLogger('logs', name='MNIST')


Ahora estamos **listos para comenzar**. El paso final es proporcionar los **datos de entrenamiento** y ajustar el modelo.  

Desactivamos la **barra de progreso** a continuación para evitar salidas extensas en el navegador durante la ejecución.


In [41]:
mnist_trainer = Trainer(deterministic=True,
                        max_epochs=30,
                        logger=mnist_logger,
                        enable_progress_bar=False,
                        callbacks=[ErrorTracker()])
mnist_trainer.fit(mnist_module,
                  datamodule=mnist_dm)


Testing: |          | 0/? [00:01<?, ?it/s]        


RuntimeError: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility

Hemos **suprimido la salida** aquí, la cual consiste en un reporte de progreso del ajuste del modelo, agrupado por **época**. Esto es muy útil, ya que en conjuntos de datos grandes el ajuste puede tomar tiempo.  

En este caso, especificamos una **división de validación del 20%**, por lo que el entrenamiento se realiza en el **80%** de las **60,000 observaciones** del conjunto de entrenamiento. Esto es una alternativa a proporcionar datos de validación explícitos, como hicimos con los datos de **`Hitters`**.

**SGD** utiliza **lotes de 256 observaciones** al calcular el gradiente, y haciendo las cuentas, observamos que una **época** corresponde a **188 pasos de gradiente**.


**`SimpleModule.classification()`** incluye por defecto una métrica de **precisión** (*accuracy*).  
Otras métricas de clasificación pueden añadirse utilizando **`torchmetrics`**.

Usaremos nuestra función **`summary_plot()`** para mostrar la **precisión** a lo largo de las **épocas**.


In [42]:
mnist_results = pd.read_csv(mnist_logger.experiment.metrics_file_path)
fig, ax = subplots(1, 1, figsize=(6, 6))
summary_plot(mnist_results,
             ax,
             col='accuracy',
             ylabel='Accuracy')
ax.set_ylim([0.5, 1])
ax.set_ylabel('Accuracy')
ax.set_xticks(np.linspace(0, 30, 7).astype(int));


FileNotFoundError: [Errno 2] No such file or directory: 'logs/MNIST/version_1/metrics.csv'

Nuevamente evaluamos la **precisión** utilizando el método **`test()`** de nuestro **trainer**.  
Este modelo alcanza un **97% de precisión** en los datos de prueba.


In [43]:
mnist_trainer.test(mnist_module,
                   datamodule=mnist_dm)

────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.08829999715089798
        test_loss            2.305781841278076
────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 2.305781841278076, 'test_accuracy': 0.08829999715089798}]

Aunque podríamos usar la función **`LogisticRegression()`** de **`sklearn`** para ajustar la regresión logística multiclase, aquí ajustaremos utilizando **`torch`**.  

Solo necesitamos una **capa de entrada** y una **capa de salida** (y omitimos las capas ocultas)


In [44]:
class MNIST_MLR(nn.Module):
    def __init__(self):
        super(MNIST_MLR, self).__init__()
        self.linear = nn.Sequential(nn.Flatten(),
                                    nn.Linear(784, 10))
    def forward(self, x):
        return self.linear(x)

mlr_model = MNIST_MLR()
mlr_module = SimpleModule.classification(mlr_model,
                                         num_classes=10)
mlr_logger = CSVLogger('logs', name='MNIST_MLR')

In [45]:
mlr_trainer = Trainer(deterministic=True,
                      max_epochs=30,
                      enable_progress_bar=False,
                      callbacks=[ErrorTracker()])
mlr_trainer.fit(mlr_module, datamodule=mnist_dm)

/home/ealvnrz/.pyenv/versions/3.10.5/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


RuntimeError: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility

Ajustamos el modelo **de la misma manera** que antes y calculamos los resultados en el conjunto de prueba.


In [46]:
mlr_trainer.test(mlr_module,
                 datamodule=mnist_dm)

────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.05660000070929527
        test_loss            2.34493350982666
────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 2.34493350982666, 'test_accuracy': 0.05660000070929527}]

La **precisión** supera el **90%** incluso para este modelo relativamente **simple**.

Al igual que en el ejemplo de **`Hitters`**, eliminamos algunos de los **objetos** que creamos anteriormente.


In [47]:
del(mnist_test,
    mnist_train,
    mnist_model,
    mnist_dm,
    mnist_trainer,
    mnist_module,
    mnist_results,
    mlr_model,
    mlr_module,
    mlr_trainer)

NameError: name 'mnist_results' is not defined

## Redes Neuronales Convolucionales (CNN)  

En esta sección, ajustamos una **CNN** a los datos de **`CIFAR100`**, los cuales están disponibles en el paquete **`torchvision`**.  

Los datos están organizados de una manera **similar** a los datos de **`MNIST`**.


In [ ]:
(cifar_train, 
 cifar_test) = [CIFAR100(root="data",
                         train=train,
                         download=True)
             for train in [True, False]]

In [48]:
transform = ToTensor()
cifar_train_X = torch.stack([transform(x) for x in
                            cifar_train.data])
cifar_test_X = torch.stack([transform(x) for x in
                            cifar_test.data])
cifar_train = TensorDataset(cifar_train_X,
                            torch.tensor(cifar_train.targets))
cifar_test = TensorDataset(cifar_test_X,
                            torch.tensor(cifar_test.targets))

NameError: name 'cifar_train' is not defined

El conjunto de datos **`CIFAR100`** consiste en **50,000 imágenes de entrenamiento**, cada una representada por un tensor tridimensional:  

- Cada imagen a **tres colores** se representa como un conjunto de **tres canales**, donde cada canal consiste en **$32 \times 32$** píxeles de **8 bits**.  

Estandarizamos los datos, como lo hicimos con los dígitos, pero **mantenemos la estructura de la matriz**. Esto se logra con la transformación **`ToTensor()`**.

La creación del **módulo de datos** es similar al ejemplo de **`MNIST`**.


In [50]:
cifar_dm = SimpleDataModule(cifar_train,
                            cifar_test,
                            validation=0.2,
                            num_workers=max_num_workers,
                            batch_size=128)


Nuevamente observamos la **forma** de los lotes típicos en nuestros **data loaders**.


In [49]:
for idx, (X_ ,Y_) in enumerate(cifar_dm.train_dataloader()):
    print('X: ', X_.shape)
    print('Y: ', Y_.shape)
    if idx >= 1:
        break


NameError: name 'cifar_dm' is not defined

Antes de comenzar, observamos algunas de las **imágenes de entrenamiento**

El ejemplo a continuación también ilustra que los objetos **`TensorDataset`** pueden ser indexados con **enteros** — estamos seleccionando imágenes **aleatorias** del conjunto de datos de entrenamiento indexando **`cifar_train`**.  

Para que las imágenes se muestren correctamente, debemos **reordenar las dimensiones** utilizando una llamada a **`np.transpose()`**.


In [50]:
fig, axes = subplots(5, 5, figsize=(10,10))
rng = np.random.default_rng(4)
indices = rng.choice(np.arange(len(cifar_train)), 25,
                     replace=False).reshape((5,5))
for i in range(5):
    for j in range(5):
        idx = indices[i,j]
        axes[i,j].imshow(np.transpose(cifar_train[idx][0],
                                      [1,2,0]),
                                      interpolation=None)
        axes[i,j].set_xticks([])
        axes[i,j].set_yticks([])


NameError: name 'cifar_train' is not defined

Aquí el método **`imshow()`** reconoce, a partir de la **forma de su argumento**, que se trata de un **arreglo tridimensional**, donde la última dimensión indexa los tres **canales de color RGB**.

Especificamos una **CNN de tamaño moderado** a modo de ejemplo.

Utilizamos varias **capas**, cada una de las cuales consiste en los pasos de **convolución**, **ReLU** y **max-pooling**.  

Primero definimos un **módulo** que representa una de estas capas. Al igual que en nuestros ejemplos anteriores, sobreescribimos los métodos **`__init__()`** y **`forward()`** de **`nn.Module`**.  

Este módulo definido por el usuario puede ahora utilizarse de la misma manera que **`nn.Linear()`** o **`nn.Dropout()`**.


In [53]:
class BuildingBlock(nn.Module):

    def __init__(self,
                 in_channels,
                 out_channels):

        super(BuildingBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels,
                              out_channels=out_channels,
                              kernel_size=(3,3),
                              padding='same')
        self.activation = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=(2,2))

    def forward(self, x):
        return self.pool(self.activation(self.conv(x)))


Notamos que utilizamos el argumento **`padding = "same"`** en **`nn.Conv2d()`**, lo que garantiza que los **canales de salida** tengan las **mismas dimensiones** que los **canales de entrada**.  

Hay **32 canales** en la primera **capa oculta**, en contraste con los **tres canales** de la **capa de entrada**. Utilizamos un **filtro de convolución de $3 \times 3$** para cada canal en todas las capas. Cada convolución es seguida por una **capa de max-pooling** sobre bloques de **$2 \times 2$**.

Al formar nuestro **modelo de aprendizaje profundo** para los datos de **`CIFAR100`**, utilizamos varios de nuestros módulos **`BuildingBlock()`** de manera **secuencial**.  

Este ejemplo simple ilustra parte del **poder de `torch`**. Los usuarios pueden **definir sus propios módulos**, los cuales pueden combinarse en **otros módulos**.  

Finalmente, todo se ajusta mediante un **entrenador genérico**.


In [51]:
class CIFARModel(nn.Module):

    def __init__(self):
        super(CIFARModel, self).__init__()
        sizes = [(3,32),
                 (32,64),
                 (64,128),
                 (128,256)]
        self.conv = nn.Sequential(*[BuildingBlock(in_, out_)
                                    for in_, out_ in sizes])

        self.output = nn.Sequential(nn.Dropout(0.5),
                                    nn.Linear(2*2*256, 512),
                                    nn.ReLU(),
                                    nn.Linear(512, 100))
    def forward(self, x):
        val = self.conv(x)
        val = torch.flatten(val, start_dim=1)
        return self.output(val)


Construimos el **modelo** y observamos el **resumen**. 


In [52]:
cifar_model = CIFARModel()
summary(cifar_model,
        input_data=X_,
        col_names=['input_size',
                   'output_size',
                   'num_params'])

NameError: name 'BuildingBlock' is not defined

El número total de **parámetros entrenables** es **964,516**.  

Al estudiar el **tamaño de los parámetros**, observamos que los **canales se reducen a la mitad** en ambas dimensiones después de cada una de estas **operaciones de max-pooling**.  

Después de la última operación, tenemos una **capa con 256 canales** de dimensión **$2 \times 2$**.  
Estas dimensiones luego se **aplanan** (*flatten*) en una **capa densa** de tamaño **1,024**; es decir, cada una de las matrices **$2 \times 2$** se convierte en un **vector de 4 elementos** y se colocan **lado a lado** en una sola capa.  

A esto le sigue una capa de **regularización por dropout**, luego otra **capa densa** de tamaño **512**, y finalmente, la **capa de salida**.

Hasta ahora, hemos estado utilizando un **optimizador predeterminado** en **`SimpleModule()`**. Para estos datos, los experimentos muestran que una **tasa de aprendizaje más pequeña** funciona mejor que el valor predeterminado de **0.01**.  
Aquí utilizamos un **optimizador personalizado** con una **tasa de aprendizaje de 0.001**.  

Además de esto, el **registro** (*logging*) y el **entrenamiento** siguen un patrón similar al de nuestros ejemplos anteriores.  

El optimizador toma un argumento **`params`**, que informa al optimizador qué **parámetros** están involucrados en **SGD** (*stochastic gradient descent*).

Vimos anteriormente que las entradas de los **parámetros de un módulo** son **tensores**. Al pasar los parámetros al optimizador, estamos haciendo más que simplemente pasar **arreglos**; parte de la **estructura del grafo** está codificada en los **tensores** mismos.


In [53]:
cifar_optimizer = RMSprop(cifar_model.parameters(), lr=0.001)
cifar_module = SimpleModule.classification(cifar_model,
                                    num_classes=100,
                                    optimizer=cifar_optimizer)
cifar_logger = CSVLogger('logs', name='CIFAR100')


NameError: name 'cifar_model' is not defined

In [54]:
cifar_trainer = Trainer(deterministic=True,
                        max_epochs=30,
                        logger=cifar_logger,
                        enable_progress_bar=False,
                        callbacks=[ErrorTracker()])
cifar_trainer.fit(cifar_module,
                  datamodule=cifar_dm)


NameError: name 'cifar_logger' is not defined

Este modelo puede tomar **10 minutos o más** en ejecutarse y alcanza aproximadamente un **42% de precisión** en los datos de prueba.  

Aunque esto no es terrible para datos con **100 clases** (un clasificador aleatorio obtiene **1% de precisión**), al buscar en la web observamos resultados cercanos al **75%**.  
Típicamente, lograr tales resultados requiere una gran cantidad de **ajustes a la arquitectura**, **regulación** (*regularization*) y **tiempo**.


Revisemos la **precisión de validación** y **entrenamiento** a lo largo de las **épocas**.


In [55]:
log_path = cifar_logger.experiment.metrics_file_path
cifar_results = pd.read_csv(log_path)
fig, ax = subplots(1, 1, figsize=(6, 6))
summary_plot(cifar_results,
             ax,
             col='accuracy',
             ylabel='Accuracy')
ax.set_xticks(np.linspace(0, 10, 6).astype(int))
ax.set_ylabel('Accuracy')
ax.set_ylim([0, 1]);

NameError: name 'cifar_logger' is not defined

Finalmente, evaluamos nuestro modelo en los **datos de prueba**.


In [56]:
cifar_trainer.test(cifar_module,
                   datamodule=cifar_dm)


NameError: name 'cifar_trainer' is not defined

### Aceleración por Hardware  

A medida que el **aprendizaje profundo** se ha vuelto ampliamente usado en el aprendizaje automático, los **fabricantes de hardware** han desarrollado **librerías especiales** que pueden acelerar los pasos de **descenso por gradiente**.

Los principales cambios ocurren en la llamada a **`Trainer()`** y en las **métricas** que se evaluarán en los datos. Estas métricas deben saber **dónde** estarán ubicados los datos en el momento de la evaluación.  

Esto se logra mediante una llamada al método **`to()`** de las métricas.


In [57]:
try:
    for name, metric in cifar_module.metrics.items():
        cifar_module.metrics[name] = metric.to('mps')
    cifar_trainer_mps = Trainer(accelerator='mps',
                                deterministic=True,
                                enable_progress_bar=False,
                                max_epochs=30)
    cifar_trainer_mps.fit(cifar_module,
                          datamodule=cifar_dm)
    cifar_trainer_mps.test(cifar_module,
                          datamodule=cifar_dm)
except:
    pass

Esto proporciona una **aceleración de aproximadamente dos o tres veces** por cada **época**.

Hemos protegido este bloque de código utilizando las cláusulas **`try:`** y **`except:`**.  
- Si funciona, obtenemos el **aumento de velocidad**.  
- Si falla, **no sucede nada**.
